In [7]:
import yfinance as yf
import pandas as pd
from pathlib import Path
import math 

In [8]:
result_folder = Path.cwd().parent / 'data'

symbols = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker'].tolist()
symbols = pd.Series(symbols).unique()
backup = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker']
if len(backup) > 0:
    backup.to_csv(result_folder / 'known_tickers.csv', index=False)
owned_tickers = pd.read_csv(result_folder / 'owned_tickers.csv')['Ticker'].tolist()

In [9]:
df = pd.DataFrame(columns=[
    'Ticker', 
    'short_name', 
    'in_portfolio',
    'value_score',  
    'recommendation_score',
    'analyst_mean%', 
    'insider_buy%', 
    'sector', 
    'industry', 
    'country',
    'P/E (TTM)', 
    'ROA%',
])

for symbol in symbols: 
    ticker = yf.Ticker(symbol)
    try:
        in_portfolio = 'No'
        if symbol in owned_tickers:
            in_portfolio = 'Yes'

        analyst_mean = 0
        if 'mean' in ticker.analyst_price_targets and ticker.info['currentPrice']:
            analyst_mean = ticker.analyst_price_targets['mean'] / ticker.info['currentPrice'] - 1
        
        roa = ticker.info['returnOnAssets']*100
        pe = ticker.info['trailingPE']
        
        value_score = format(2.5 + math.log(roa+5) - math.log(pe+25), '.2f')
        
        insider_buy = 0
        if 'Shares' in ticker.insider_purchases.columns:
            insider_buy = format(ticker.insider_purchases.loc[ticker.insider_purchases.index[4], 'Shares'], '.2f')
            if insider_buy == '<NA>':
                insider_buy = 0

        recommendation_score = format(float(value_score) + analyst_mean * 1.25 + float(insider_buy) * 2.0, '.2f')

        df.loc[symbol] = [
            symbol, 
            ticker.info['shortName'], 
            in_portfolio,
            value_score,
            recommendation_score,
            format(analyst_mean * 100, '.2f'), 
            insider_buy,
            ticker.info['sector'], 
            ticker.info['industry'], 
            ticker.info['country'],
            format(pe, '.2f') if isinstance(pe, (int, float)) else 'N/A', 
            format(roa, '.2f') if isinstance(roa, (int, float)) else 'N/A',
        ]
    except Exception as e:
        print(f"Error with {symbol}: {e}. Removed Ticker from list.")

df.to_csv(result_folder / 'simple_screener_results.csv', index=False)
display(df)

print(f'Got data for {len(df)} stocks')

,Ticker,short_name,in_portfolio,value_score,recommendation_score,analyst_mean%,insider_buy%,sector,industry,country,P/E (TTM),ROA%
MATAS.CO,MATAS.CO,Matas A/S,No,0.81,1.12,25.00,0,Consumer Cyclical,Specialty Retail,Denmark,22.82,3.84
TRIFOR.CO,TRIFOR.CO,Trifork Group AG,No,0.72,1.06,27.20,0,Technology,Information Technology Services,Switzerland,14.03,1.59
QQ.L,QQ.L,QINETIQ GROUP PLC ORD 1P,Yes,1.25,1.50,20.13,0,Industrials,Aerospace & Defense,United Kingdom,16.70,6.89
RNMBY,RNMBY,Rheinmetall AG,Yes,0.21,0.21,0.00,0,Industrials,Aerospace & Defense,Germany,91.49,6.81
SAABF,SAABF,Saab AB,Yes,0.31,0.31,0.00,0,Industrials,Aerospace & Defense,Sweden,54.04,3.80
BCKIY,BCKIY,Babcock International Group Plc,No,0.93,0.93,0.00,0,Industrials,Engineering & Construction,United Kingdom,23.60,5.08
BAESY,BAESY,BAE Systems PLC,No,0.77,0.54,-18.67,0,Industrials,Aerospace & Defense,United Kingdom,28.30,4.40
IVSO.ST,IVSO.ST,INVISIO AB,No,1.30,1.22,-6.13,0,Technology,Communication Equipment,Sweden,58.94,20.31
NSKFF,NSKFF,Kongsberg Gruppen ASA,Yes,0.65,0.65,0.00,0,Industrials,Aerospace & Defense,Norway,48.92,6.65
GMAB,GMAB,Genmab A/S,Yes,1.66,2.12,36.67,0,Healthcare,Biotechnology,Denmark,11.73,10.91


Got data for 36 stocks
